In [13]:
import os 
import json
import pandas as pd


train_f = open(os.path.join("data", "raw", f"train_with-reference.jsonl"), "r")
data = []
for line in train_f:
    data.append(json.loads(line))
train_f.close()

dev_f = open(os.path.join("data", "raw", f"dev_without-reference.jsonl"), "r")
dev_data = []
for line in dev_f:
    dev_data.append(json.loads(line))
dev_f.close()

In [14]:
import random
from itertools import tee

def triplewise(iterable):
    a, b, c = tee(iterable, 3)
    next(b, None)
    next(c, None)
    next(c, None)
    return zip(a, b, c)

def get_trigrams(convo):
    texts = [utterance['text'] for utterance in convo['utterances']]
    trigrams = set(triplewise(texts))
    return trigrams

def has_overlap(convo1, convo2):
    convo1_trigrams = get_trigrams(convo1)
    convo2_trigrams = get_trigrams(convo2)

    return bool(convo1_trigrams.intersection(convo2_trigrams))

random.shuffle(data)
test_size = int(0.1 * len(data))

train_data = data[test_size:]
test_data = data[:test_size]

for convo_test in test_data:
    for convo_train in train_data:
        if has_overlap(convo_test, convo_train):
            train_data.remove(convo_train)
            test_data.append(convo_train)

print("Train set size:", len(train_data))
print("Test set size:", len(test_data))

Train set size: 1857
Test set size: 890


In [24]:
# check overlap between train and dev
count = 0
ids = set()
for convo in train_data:
    for train_convo in train_data:
        if train_convo != convo and has_overlap(convo, train_convo):
            count += 1
            ids.add(convo['id'])
    #         print("Overlap found!", convo['id'], train_convo['id'])
    #         # print the 2 utterances and the 3 utterances that overlap
    #         print ("Convo 1:", convo['utterances'])
    #         print ("Convo 2:", train_convo['utterances'])
    #         print ("Overlap:", get_trigrams(convo).intersection(get_trigrams(train_convo)))
    #         break
    # break
print ("Number of overlaps between train and dev:", len(ids))

Number of overlaps between train and dev: 1236
